In [ ]:
# make a data generator that yeild the features and labels
# we first need to cut the audio into frames
# then get the labels to fit those frames

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# cut audio into frames
import os

import numpy as np
import pandas as pd


from pitch_tracker.utils.constants import (F_MIN, HOP_LENGTH, N_FFT, N_MELS,
                                           PICKING_FRAME_SIZE,
                                           PICKING_FRAME_STEP,
                                           PICKING_FRAME_TIME, SAMPLE_RATE,
                                           STEP_FRAME, STEP_TIME, WIN_LENGTH,
                                           N_CLASS, )
from pitch_tracker.utils.audio import load_audio_mono
from pitch_tracker.utils.files import get_file_name, list_file_paths_in_dir

from pitch_tracker.utils import dataset

In [ ]:
audio_path = '../medleydb/medleydb/data/Audio/Phoenix_ScotchMorris/Phoenix_ScotchMorris_MIX.wav'
signal, sample_rate = load_audio_mono(audio_path, SAMPLE_RATE, keep_channel_dim=True)
# signal = torch.unsqueeze(signal,0)
# signal = torch.from_numpy(signal)
mel_features = dataset.extract_melspectrogram_feature(signal,N_FFT, HOP_LENGTH, N_MELS, SAMPLE_RATE, backend='librosa')
print(mel_features.shape)

stft_feature = dataset.extract_stft_feature(signal, N_FFT, HOP_LENGTH)
print(stft_feature.shape)

In [ ]:
csv_path = '../content/gen_label/2560/Melody2_midi/AClassicEducation_NightOwl.csv'
csv_folder = '../content/gen_label/2560/Melody2_midi/'
label_dict = dataset.create_label_dict_from_dir(csv_folder)
for k,v in label_dict.items():
    print(k,v.shape)
    

In [ ]:
dataset_paths = dataset.create_dataset_path_dict(csv_folder)
dataset_paths_df = pd.DataFrame(dataset_paths).transpose()
# dataset_paths_df.columns=['track_id', 'label_path', 'audio_path']
display(dataset_paths_df.head(5))
del dataset_paths_df

In [ ]:
feature_label_gen = dataset.create_feature_label_generator(
    dataset_path_dict=dataset_paths, 
    sample_rate= SAMPLE_RATE,
    n_fft= N_FFT,
    n_mels= N_MELS,
    n_class= N_CLASS,
    hop_length= HOP_LENGTH,
    picking_frame_step= PICKING_FRAME_STEP,
    picking_frame_size= PICKING_FRAME_SIZE,
    step_frame= STEP_FRAME,
    step_time= STEP_TIME,
    dist_threshold= 0.1,
    empty_threshold= 0.3,
)
passed_songs = []
for song_name, feature_label_pair in feature_label_gen:
    n_big_frame = 0
    passed_songs.append(song_name)
    for feature, label in feature_label_pair:
        # print(feature.shape)
        # print(label[0].shape)
        # print(label[1].shape)
        # print(label[2].shape)
        n_big_frame+=1
    print(song_name, n_big_frame)

In [ ]:
print(label[0])

In [ ]:
missing_label = []
for label in dataset_paths:
    if label not in passed_songs:
        missing_label.append(label)
missing_label